# Ejemplo Procesamiento de imágenes

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 10 de marzo de 2025

En este ejemplo se muestra como abrir una imagen y aplicar un filtro usando OpenCV.

Requiere:
```
pip install opencv-contrib-python pyqt5 matplotlib
```

## Definir funciones auxiliares

Se usa un diálogo en QT para seleccionar la imagen a procesar.

In [ ]:
import sys
import os
import numpy
import cv2
from PyQt5.QtWidgets import QApplication, QFileDialog
import matplotlib.pyplot as plt


def mostrar_imagen(window_name, imagen):
    # si la imagen a mostrar es muy grande reducir su tamaño
    MAX_WIDTH = 700
    MAX_HEIGHT = 500
    if imagen.shape[0] > MAX_HEIGHT or imagen.shape[1] > MAX_WIDTH:
        fh = MAX_HEIGHT / imagen.shape[0]
        fw = MAX_WIDTH / imagen.shape[1]
        escala = min(fh, fw)
        imagen = cv2.resize(imagen, (0, 0), fx=escala, fy=escala)
    # mostrar en pantalla
    cv2.imshow(window_name, imagen)


def ui_select_filenames():
    # mostrar un dialogo de seleccionar un archivo
    app = QApplication(list());
    options = QFileDialog.Options()
    files, _ = QFileDialog.getOpenFileNames(None, "Imagenes", ".", "Imagenes (*.jpg *.png)", options=options)
    return files


def abrir_imagen(filename):
    imagen_color = cv2.imread(filename, cv2.IMREAD_COLOR)
    if imagen_color is None:
        raise Exception("error abriendo {}".format(filename))
    return imagen_color


def histograma_gris(imagen_8bits):
    bins = range(0, 257, 1)
    xticks = list(range(0, 256, 64))
    xticks.append(255)
    plt.hist(imagen_8bits.reshape(-1), bins=bins, density=True)
    plt.xticks(xticks)
    plt.xlabel('grises')
    plt.ylabel('cantidad')
    plt.title('Histograma de intensidades')
    plt.show()


print("Usando OpenCV {} con Python {}.{}.{}".format(cv2.__version__, sys.version_info.major, sys.version_info.minor,
                                                    sys.version_info.micro))


## Ejemplo 1 - Histograma y Ecualización

In [ ]:
def ejemplo(filename):
    imagen_color = abrir_imagen(filename)
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)
    imagen_eq = cv2.equalizeHist(imagen_gris)
    mostrar_imagen(filename + " (gris)", imagen_gris)
    mostrar_imagen(filename + " (eq)", imagen_eq)
    histograma_gris(imagen_gris)
    histograma_gris(imagen_eq)


filenames = ui_select_filenames()

if len(filenames) > 0:
    for filename in filenames:
        ejemplo(filename)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

print("FIN")

## Ejemplo 2 - Binarización con OTSU

In [ ]:
def ejemplo(filename):
    imagen_color = abrir_imagen(filename)
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)
    threshold, imagen_bin = cv2.threshold(imagen_gris, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    mostrar_imagen(filename, imagen_color)
    mostrar_imagen(filename + " (gris)", imagen_gris)
    mostrar_imagen(filename + " (bin)", imagen_bin)
    print("{} size={} threshold={}".format(filename, imagen_color.shape, threshold))
    histograma_gris(imagen_gris)
    histograma_gris(imagen_bin)


filenames = ui_select_filenames()

if len(filenames) > 0:
    for filename in filenames:
        ejemplo(filename)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

print("FIN")

## Ejemplo 3 - Convolución

Revisar el tratamiento del borde de la función **filter2d** [[en este link]](https://docs.opencv.org/4.x/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04).

Por defecto usa borde tipo `BORDER_REFLECT_101` que la documentación describe así:

`gfedcb|`**`abcdefgh`**`|gfedcba`

es decir, para una imagen que en una fila tiene pixeles `abcdefgh`, crea pixeles antes y después como si estuvieran en un espejo.

Se pueden ver otros parámetros para el borde en: https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga209f2f4869e304c82d07739337eae7c5

Notar que, en general, no es muy relevante definir qué hacer en el borde ya que afecta unos pocos pixeles. Pero sí se vuelve relevante cuando las imágenes son muy pequeñas o cuando contienen algún patrón especial.


In [ ]:
def ejemplo(filename):
    imagen_color = abrir_imagen(filename)
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)
    imagen_gaussian = cv2.GaussianBlur(imagen_gris, (9, 9), 0)
    imagen_median = cv2.medianBlur(imagen_gris, 9)
    kernel = numpy.array([[1, 1, 1],
                          [1, 1, 1],
                          [1, 1, 1]]) / 9
    # una forma alternativa para probar kernels más grandes:
    # kernel = numpy.ones((15, 15), numpy.float32) / (15*15)
    imagen_conv = cv2.filter2D(imagen_gris, -1, kernel)
    mostrar_imagen("gris: " + filename, imagen_gris)
    mostrar_imagen("gaussian: " + filename, imagen_gaussian)
    mostrar_imagen("median: " + filename, imagen_median)
    mostrar_imagen("convolucion: " + filename, imagen_conv)


filenames = ui_select_filenames()

if len(filenames) > 0:
    for filename in filenames:
        ejemplo(filename)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

print("FIN")

Más detalles y ejemplos se pueden ver en la página de OpenCV: https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html 